In [19]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Some hyperparameters and settings

In [21]:
CONFIG_FILE_PATH = 'config/sdd_longterm.yaml'  # yaml config file containing all the hyperparameters
DATASET_NAME = 'sdd'

TEST_DATA_PATH = 'data/SDD/test_longterm.pkl'
TEST_IMAGE_PATH = 'data/SDD/test'
OBS_LEN = 5  # in timesteps
PRED_LEN = 30  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

ROUNDS = 3  # Y-net is stochastic. How often to evaluate the whole dataset
BATCH_SIZE = 8

#### Load config file and print hyperparameters

In [22]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'resize': 0.25,
 'batch_size': 8,
 'viz_epoch': 10,
 'learning_rate': 0.0001,
 'num_epochs': 300,
 'encoder_channels': [32, 32, 64, 64, 64],
 'decoder_channels': [64, 64, 64, 32, 32],
 'waypoints': [14, 29],
 'temperature': 1.8,
 'segmentation_model_fp': 'segmentation_models/SDD_segmentation.pth',
 'semantic_classes': 6,
 'loss_scale': 1000,
 'kernlen': 31,
 'nsig': 4,
 'use_features_only': False,
 'unfreeze': 100,
 'use_TTST': True,
 'rel_threshold': 0.002,
 'use_CWS': True,
 'CWS_params': {'sigma_factor': 6, 'ratio': 2, 'rot': True}}

#### Load preprocessed Data

In [23]:
df_test = pd.read_pickle(TEST_DATA_PATH)

In [24]:
df_test.head()

,trackId,frame,x,y,sceneId,metaId
0,5,311,1608.0,326.5,coupa_0,0
1,5,341,1608.0,326.5,coupa_0,0
2,5,371,1608.0,326.5,coupa_0,0
3,5,401,1608.0,326.5,coupa_0,0
4,5,431,1608.0,326.5,coupa_0,0


#### Initiate model and load pretrained weights

In [28]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

ModuleNotFoundError: No module named 'segmentation_models_pytorch'

In [26]:
model.load(r'D:\AKG\GSN\Human-Path-Prediction\ynet\pretrained_models\sdd_longterm_weights.pt')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\adeel\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\adeel\AppData\Local\Temp\ipykernel_1664\2962444527.py", line 1, in <cell line: 1>
    model.load(r'D:\AKG\GSN\Human-Path-Prediction\ynet\pretrained_models\sdd_longterm_weights.pt')
  File "d:\AKG\GSN\Human-Path-Prediction\ynet\model.py", line 417, in load
    print(self.model.load_state_dict(torch.load(path)))
  File "c:\Users\adeel\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py", line 1671, in load_state_dict
    raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
RuntimeError: Error(s) in loading state_dict for YNetTorch:
	Unexpected key(s) in state_dict: "semantic_segmentation.encoder.conv1.weight", "semantic_segmentation.encoder.bn1.weight", "semantic_segmentation.encoder.bn1.bias",

#### Evaluate model

In [27]:
model.evaluate(df_test, params, image_path=TEST_IMAGE_PATH,
               batch_size=BATCH_SIZE, rounds=ROUNDS, 
               num_goals=NUM_GOALS, num_traj=NUM_TRAJ, device=None, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|██████████| 11/11 [00:00<00:00, 1075.39it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (11,) + inhomogeneous part.